In [229]:
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import collections
from numpy.linalg import inv

In [248]:
def get_max_arg(g_fn):
    
    max_arg = 0
    sum_list = [[] for i in range(len(g_fn))]
    
    for index_i, value_i in enumerate(g_fn):
        sum_val = 0
        for index_j, value_j in enumerate(value_i):
            sum_val += value_j
        sum_list[index_i].append(sum_val)
    
    max_sum = sum_list[0]
    max_index = 0
    for index_k, value_k in enumerate(sum_list):
#         print(value_k)
        if value_k > max_sum:
            max_sum = value_k
            max_index = index_k
    
    return max_index


def LDA(X, predict_x):
    
    ###
    
    N_g_list = []
    for i in range(len(X)):
        ind = int(X[i][-1])
        N_g_list.append(ind)
    N_g = collections.Counter(N_g_list)
    
    ###
    
    p_g = []
    for i in (N_g):
        p_g.append((N_g[i])/len(N_g_list))

    ###
    
    classes = []

    for i in (N_g):
        classes.append(i)

    ###
    
    x_y_split = [] 
    x_list = [[] for i in range(len(N_g))]

    for i in range(len(X)):
        x_y_split.append(np.split(X[i], [-1]))

    for j in range(len(X)):
        for k in range(len(classes)):
            if x_y_split[j][1] == classes[k]:
                x_list[k].append(x_y_split[j][0])

    x_list_store = x_list
    
    ###
    
    mean_list = []

    for counter, value in enumerate(N_g):
        mean = (1/N_g[value]) * (sum(x_list[counter]))
        mean_list.append(mean)

    ###
    
    coeff = 0

    x_list = x_list_store

    mean_list = np.array(mean_list)

    denominator = len(X) - len(classes)

    numerator = 0

    for class_index, class_value in enumerate(classes):
        x_minus_mean = []

        for x_val in x_list[class_index]:
            temp = x_val - mean_list[class_index]
            x_minus_mean.append(temp)

        x_minus_mean = np.array(x_minus_mean)
        numerator += np.dot(x_minus_mean.T, x_minus_mean)

    coeff = (numerator/denominator)
    
    ### 
    
    g_fn = []

#     predict_x = [0.5]
    predict_x = np.array(predict_x)

    for class_index, class_value in enumerate(classes):
        _a = (np.log(p_g[class_index]))
        _b = np.dot(np.dot(predict_x,inv(coeff)),mean_list[class_index])
        _c = -(0.5*np.dot(np.dot((mean_list[class_index].T),(inv(coeff))), mean_list[class_index]))
        g_fn_class = _a + _b + _c
#         g_fn_class = (np.log(p_g[class_index])) + ((predict_x.T)*(inv(coeff))*mean_list[class_index]) - (0.5*(mean_list[class_index].T)*(inv(coeff))*mean_list[class_index])
        g_fn.append(g_fn_class)
    
    g_fn = np.array(g_fn)

#     argmax_val = get_max_arg(g_fn)
    
    print(g_fn)
    argmax_val = g_fn.argmax()
    
#     print("g_fn:", g_fn)
#     print("argmax:", argmax_val)
    
    return (classes[argmax_val])

In [249]:
N = 30

np.random.seed(123)
X1 = np.random.multivariate_normal([1,1],[[1,0],[0,1]],N)
X1 = np.concatenate((X1,1.0*np.ones((N,1))),axis=1)
X2 = np.random.multivariate_normal([-1,-1],[[1,0],[0,1]],N)
X2 = np.concatenate((X2,2.0*np.ones((N,1))),axis=1)
X3 = np.random.multivariate_normal([2,0],[[1,0],[0,1]],N)
X3 = np.concatenate((X3,3.0*np.ones((N,1))),axis=1)

X = np.concatenate((X1,X2,X3));

In [250]:
# X = np.array([(0,1), (1,2), (1.5,2), (2,2)])
# x_pred = [0.5]

# print("Predicted Class:", LDA(X, x_pred))

In [252]:
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = LDA(X, np.c_[xx.ravel(), yy.ravel()])
# Z = Z.reshape(xx.shape)
plt.figure(dpi=300)
plt.pcolormesh(xx, yy, Z, cmap='Pastel2', shading='auto')

plt.scatter(X[:,0],X[:,1],c=X[:,2],cmap='Set2');
plt.title("Linear classifier (linear discriminant analysis)")
plt.show()

[[ -9.59437825  -9.57461306  -9.55484787 ...   5.99178519   6.01155038
    6.03131557]
 [  5.10068962   5.08649914   5.07230866 ...  -9.77444078  -9.78863126
   -9.80282174]
 [-11.05776972 -11.0172711  -10.97677249 ...   5.34583285   5.38633147
    5.42683009]]


IndexError: list index out of range

## Step by step process

In [119]:
N_g_list = []

for i in range(len(X)):
    ind = int(X[i][-1])
    N_g_list.append(ind)
    
N_g = collections.Counter(N_g_list)
print(N_g)

Counter({2: 3, 1: 1})


In [120]:
p_g = []

for i in (N_g):
    p_g.append((N_g[i])/len(N_g_list))

print(p_g)

[0.25, 0.75]


In [121]:
classes = []

for i in (N_g):
    classes.append(i)
    
print(classes)

[1, 2]


In [122]:
x_y_split = [] 
x_list = [[] for i in range(len(N_g))]

for i in range(len(X)):
    x_y_split.append(np.split(X[i], [-1]))
    
for j in range(len(X)):
    
    for k in range(len(classes)):
        
        if x_y_split[j][1] == classes[k]:
            x_list[k].append(x_y_split[j][0])

x_list_store = x_list

# print(x_y_split)
# print(x_list)
# print(x_list[0])
# print(x_list[1])

In [123]:
mean_list = []

for counter, value in enumerate(N_g):
    mean = (1/N_g[value]) * (sum(x_list[counter]))
    mean_list.append(mean)
    
print(mean_list)

[array([0.]), array([1.5])]


In [124]:
coeff = 0

x_list = x_list_store

mean_list = np.array(mean_list)

denominator = len(X) - len(classes)

numerator = 0

for class_index, class_value in enumerate(classes):
    x_minus_mean = []
    
    for x_val in x_list[class_index]:
        temp = x_val - mean_list[class_index]
        x_minus_mean.append(temp)
        
    x_minus_mean = np.array(x_minus_mean)
    numerator += np.dot(x_minus_mean.T, x_minus_mean)
    
coeff = (numerator/denominator)
print(coeff)

[[0.25]]


In [125]:
coeff_list = []

for i in range(len(classes)):
    x_minus_mean = x_list[i] - mean_list[i] 
    coeff_temp = np.dot(x_minus_mean.T, x_minus_mean)
    coeff_list.append(coeff_temp)

coeff = (1/(len(N_g_list)-len(classes)) * sum(coeff_list))
print(coeff)

[[0.25]]


In [126]:
g_fn = []

# predict value is given 
predict_x = [0.5]
predict_x = np.array(predict_x)

for class_index, class_value in enumerate(classes):
    g_fn_class = (np.log(p_g[class_index])) + ((predict_x.T)*(inv(coeff))*mean_list[class_index]) - (0.5*(mean_list[class_index].T)*(inv(coeff))*mean_list[class_index])
    g_fn.append(g_fn_class)

print(g_fn)
    
# find index of larges g_fn
# max_index = 0
# max_value = 0
# for index, item in enumerate(g_fn):
#     pass
#     print(item)
#     if item > max_value:
#         max_value = item
#         max_index = index 
print(g_fn)
g_fn = np.array(g_fn)
print(g_fn)
argmax_val = g_fn.argmax()
print(check)
print("Predicted Class = ", classes[argmax_val])

# predicted_class_output = classes[max_index]
# print(predicted_class_output)

[array([[-1.38629436]]), array([[-1.78768207]])]
[array([[-1.38629436]]), array([[-1.78768207]])]
[[[-1.38629436]]

 [[-1.78768207]]]
0
Predicted Class =  1


In [127]:
# new_X = []
# new_y = []

# for i in X:
#     temp_new_X = (i[:-1])
#     new_X.append(temp_new_X)
#     temp_new_y = (i[-1:])
#     new_y.append(temp_new_y)

# new_X = np.array(new_X)
# new_y = np.array(new_y)

In [128]:
# new_predict_x = predict_x
# new_predict_x = new_predict_x.reshape(-1,1)

# print(new_predict_x)
# print(new_predict_x.shape)

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# clf = LinearDiscriminantAnalysis()
# clf.fit(new_X, new_y)
# print(clf.predict([new_predict_x]))